# Week6 Statistical Inference with Python

In week6, we've covered:
* **Variance**:  
    * Measuring Uncertainty  
    * Prior and Posterior Distributions  
    * Credible Intervals  
* **Inference**:  
    * Hypothesis Testing  
    * False Discovery Rate  

The best way to consolidate the knowledge in your mind is by practicing.<br>Please complete the part marked with <span style="color:green">**# TODO**</span>.

[Google](www.google.com) and [Python Documentation](https://docs.python.org/3/contents.html) are your good friends if you have any python questions.

Upload **Week6_Statistical_Inference_With_Python_Homework.ipynb** notebook to your Google Drive and open it with Google Colab

## Inference  

Recall that last week, we were introduced to statistical inference using batting averages from baseball as a running example. We used the binomial distribution to model the likelihood that a player would get a hit during an at bat, given historical data. We used the beta distribution to estimate batting averages and measure our uncertainty.  

This covered the first two articles of this series: http://varianceexplained.org/r/simulation-bayes-baseball/  

Let's continue the discussion of variance, and discuss how to use variance to come up with a range for our estimates of batting averages. By the end of this exercise, you should feel comfortable giving a probability that an estimate lies in an interval. Read about credible intervals in the third article of the series: http://varianceexplained.org/r/credible_intervals_baseball/  

### Exercises  

1. Run the codeblock below to reproduce the dataset from last week.  

In [2]:
import pandas as pd  

# read the dataset  
batting_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/Batting.csv'
pitching_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/Pitching.csv'
people_url = 'https://raw.githubusercontent.com/chadwickbureau/baseballdatabank/master/core/People.csv'

batting = pd.read_csv(batting_url)
pitching = pd.read_csv(pitching_url)
master = pd.read_csv(people_url)

# recreate career dataframe
pitchers = pitching['playerID'].tolist()
batting = batting[batting['AB'] > 0]
batting = batting[~batting['playerID'].isin(pitchers)] #filtered out pitchers
batting_sum =batting.groupby(['playerID']).agg({'H':'sum','AB':'sum'}) # get total hits and at bats for each player
batting_sum['Avg'] = batting_sum.loc[:,'H'] / batting_sum.loc[:,'AB'] #calculate the avg batting rate
career = pd.merge(batting_sum, master, how='inner', on='playerID')[["playerID","nameFirst", "nameLast", "H", "AB", "Avg"]]
career= career[career["AB"]>=500]
career.head()

# helper function to estimate priors for alpha and beta
def moments(mu, sigma2):
    alpha = mu**2 * ((1 - mu) / sigma2 - 1 / mu)
    beta = alpha * (1 / mu - 1)
    return (alpha, beta)

# get priors for alpha and beta  
empirical_mean = career['Avg'].to_numpy().mean()
empirical_variance = career['Avg'].to_numpy().var()
alpha0, beta0 = moments(empirical_mean, empirical_variance)

# use priors to update estimate of batting average for each player
career['eb'] = ((career.loc[:,'H'] + alpha0) / (career.loc[:,'AB'] + alpha0 + beta0))

# concatenate first and last name
career['name'] = career.loc[:, 'nameFirst'] + ' ' + career.loc[:, 'nameLast']
career.sort_values('eb', ascending=False).loc[:,["playerID", 'name', 'H', 'AB', 'Avg', 'eb']].head(5)

,playerID,name,H,AB,Avg,eb
4106,hornsro01,Rogers Hornsby,2930,8173,0.358497,0.354854
4296,jacksjo01,Shoeless Joe Jackson,1772,4981,0.355752,0.350070
2202,delahed01,Ed Delahanty,2597,7510,0.345806,0.342354
3619,hamilbi01,Billy Hamilton,2164,6283,0.344421,0.340392
3818,heilmha01,Harry Heilmann,2660,7787,0.341595,0.338422


2. Recall `alpha0` and `beta0` represent prior belief for the "average player" as determined by the entire data set. Use your knowledge from last week to generate an array of `1000` samples from a beta distribution with parameters `alpha0` and `beta0`. 
    
 2a. Calculate the sample mean and compare it to the ratio $\frac{\alpha_0}{\alpha_0 + \beta_0}$

In [3]:
# TODO
from numpy import random
# 
n=1000
samples = random.beta(alpha0, beta0, size=n)

import statistics

print("Sample Mean:", statistics.mean(samples))
print("ALPHA0/(ALPHA0+BETA0)", alpha0/(alpha0+beta0))

Sample Mean: 0.25891472830974654
ALPHA0/(ALPHA0+BETA0) 0.25837060609701573


  2b. Calculate the 0.10 and 0.90 quantlies of the sample.  

In [4]:
# TODO
import numpy as np

print("Q .10 quantile : ", np.quantile(samples, .10))
print("Q .90 quantile : ", np.quantile(samples, .90))

Q .10 quantile :  0.2281822950411991
Q .90 quantile :  0.29162186886732555


3. Find two players in the `career` dataframe that have a similar `Avg` but very different at bats (`AB`).
  
  3a. Create a two-row dataframe that includes only these two players, and display their `name`, `H`, `AB`, `Avg`, and estimated avg (`eb`).

In [5]:
max_ab = max(career["AB"])
min_ab = min(career["AB"])

print("Max AB:", max_ab) # looking for the higher at bats 
print("Min AB:", min_ab ) # Looking for the lower

Max AB: 14053
Min AB: 500


In [6]:
print("Higher AB") # Max AB
career[career.AB == 14053]

Higher AB


,playerID,nameFirst,nameLast,H,AB,Avg,eb,name
7697,rosepe01,Pete,Rose,4256,14053,0.302853,0.301897,Pete Rose


In [7]:
#player with a similar avg
round_digit= 5
avg_max_bats = round(career[career.AB == 14053]["Avg"].values[0], round_digit)
print(career[career.AB == 14053]["Avg"].values[0])
career[(round(career["Avg"],round_digit) == avg_max_bats) & (career.AB != 14053) ]

two_players = career[(career.playerID =="swansev01") |  (career.playerID =="rosepe01")]
two_players

0.3028534832420124


,playerID,nameFirst,nameLast,H,AB,Avg,eb,name
7697,rosepe01,Pete,Rose,4256,14053,0.302853,0.301897,Pete Rose
8722,swansev01,Evar,Swanson,573,1892,0.302854,0.296615,Evar Swanson


3b. Calculate the *posterior* values for $\alpha$ and $\beta$ for the two players in your subset.

(This Stack Overflow answer details the update process for $\alpha$ and $\beta$: https://stats.stackexchange.com/a/47782)

In [8]:
# TODO
two_players= two_players.assign(alpha0 = alpha0)
two_players =two_players.assign(beta0 = beta0)

#Posterior values for the two players
two_players['alpha1'] =  two_players['alpha0'] + two_players['H'] 
two_players['beta1'] = two_players['beta0'] + (two_players['AB'] - two_players['H'])
two_players['Avg1']=(two_players['alpha1']/(two_players['alpha1']+two_players['beta1']))
pete = two_players[two_players.playerID =='rosepe01']
evar = two_players[two_players.playerID =='swansev01']
two_players

,playerID,nameFirst,nameLast,H,AB,Avg,eb,name,alpha0,beta0,alpha1,beta1,Avg1
7697,rosepe01,Pete,Rose,4256,14053,0.302853,0.301897,Pete Rose,79.746164,228.904132,4335.746164,10025.904132,0.301897
8722,swansev01,Evar,Swanson,573,1892,0.302854,0.296615,Evar Swanson,79.746164,228.904132,652.746164,1547.904132,0.296615


  3c. Though the `Avg` and the priors (`alpha0`, `beta0`) are similar for your two players, their estimated variance should be very different (due to the difference in at bats). Make an argument (no proof required): Which of your two players will have a lower 0.10 quantile for the posterior distribution of their estimated batting average (`eb`)? Explain your reasoning in a sentence or two.

TODO: Make your argument:

Q .10 quantile :  0.23812556103883104
Q .90 quantile :  0.30626701150559615

Since both values for Avg1 are below the 90, we cal 


3d. Use your posterior alphas and betas to generate a sample of `1000` estimated batting averages for each of your two players. Calculate the 0.10 and 0.90 quantile for each sample distribution. Compare the outcome to your prediction in 3c. Reconcile any differences.

In [9]:
import statistics as stats
def analyze_player(name, alpha1, beta1, size = 1000):
    sam = random.beta(alpha1,beta1, size=size)
    mean = stats.mean(sam)
    vari = stats.variance(sam)
    std_dev = stats.stdev(sam)
    print(f"{name}\n")
    print(f"Mean: {mean} \nVariance: {vari} \nStd Dev: {std_dev}\n\n")
    print(".10 quantile : ", np.quantile(sam, .10))
    print(".90 quantile : ", np.quantile(sam, .90))
    return {"samples":sam, "mean":mean, "variance":vari, "std_dev": std_dev}

In [10]:
pete_result = analyze_player("Pete", pete['alpha1'].values[0], pete['alpha1'].values[0])

Pete

Mean: 0.5000304827581709 
Variance: 2.8485731281971548e-05 
Std Dev: 0.0053372025708203685


.10 quantile :  0.4933470533838703
.90 quantile :  0.5068841300690204


In [11]:
evar_result = analyze_player("Evar", evar['alpha1'].values[0], evar['alpha1'].values[0])

Evar

Mean: 0.4993971200579186 
Variance: 0.00020448503114987496 
Std Dev: 0.014299826262926238


.10 quantile :  0.48116525480330913
.90 quantile :  0.5178737408903996


4. Finally, let's compare confidence and credible intervals.

  4a. For each of your two players, use their `H` and `AB` to calculate the 95% binomial proportion confidence interval.
  
  (The wikipedia entry will be helpful: https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval. Note that for a 95% confidence interval, the *z* score is 1.96.)

In [12]:
# TODO
import math
# Calculate interval
def print_confidence_interval(z, n, std_dev, avg, name):
    delta = z * std_dev/math.sqrt(n)
    print(f"Confidence Interval for {name} {avg - delta} to {avg + delta}")

z = 1.96
print_confidence_interval(z,pete['AB'].values[0], pete_result['std_dev'], pete['Avg'].values[0], "Pete" )
print_confidence_interval(z,evar['AB'].values[0], evar_result['std_dev'], evar['Avg'].values[0], "Evar" )

Confidence Interval for Pete 0.3027652392320932 to 0.3029417272519316
Confidence Interval for Evar 0.3022097660437286 to 0.3034984791994003


4b. Calculate the 95% credible interval using `alpha` and `beta`. Compare the difference between the confidence interval and the credible interval.

(Here is another resource for understanding confidence vs credible intervals: https://towardsdatascience.com/do-you-know-credible-interval-e5b833adf399)

In [14]:
# TODO

from scipy.special import btdtri
b_up = btdtri(alpha0, beta0, 0.975)
b_lo = btdtri(alpha0, beta0, 0.025)
print(f"credible Interval: {b_lo} to {b_up}")

credible Interval: 0.2111477481958584 to 0.30855909470110054


Confidence interval:
calculated for each one fo the two players
gives us an idea of those two player


Confidence interval:
Calculated for the complete data
gives us an idea of the average and where the 95% of the obsevations shou

## Hypothesis Testing

Now that you have a handle on credible intervals, you are ready to use those intervals to test hypotheses. 

Read the fourth article to begin to understand how credible intervals can be useful to make decisions: http://varianceexplained.org/r/bayesian_fdr_baseball/

### Exercises  

5. Let's aggregate Hank Aaron's seasons to get his total career at bats. 

In [71]:
h_aaron = career[career['playerID'] == 'aaronha01']
h_aaron.head(5)

,playerID,nameFirst,nameLast,H,AB,Avg,eb,name
0,aaronha01,Hank,Aaron,3771,12364,0.304998,0.303863,Hank Aaron


5a. Calculate Hank Aaron's total Hits (`H`) and At Bats (`AB`)

In [72]:
# TODO
#calculate??
#I understand they are given
h_aaron[['H', 'AB']]

,H,AB
0,3771,12364


5b. Use random sampling to estimate the probability that Hank Aaron's career batting average is below 0.300.

In [75]:
# TODO
a_alpha = h_aaron['H'].values[0]
a_beta = h_aaron['AB'].values[0] - h_aaron['H'].values[0]
result=analyze_player("Aaron", a_alpha, a_beta, 5000)
is_lower_than_averge = [1 if sample < .3  else 0 for sample in result['samples'] ]
print(f"Percentage of the lower than average of 0.3 of all the samples is the {sum(is_lower_than_averge)*100/len(is_lower_than_averge)} %", )

Aaron

Mean: 0.30490639563237737 
Variance: 1.606395903788106e-05 
Std Dev: 0.004007986905901897


.10 quantile :  0.29967025041312595
.90 quantile :  0.31010885810474403
Percentage of the lower than average of 0.3 of all the samples is the 11.6 %


5c. Use random sampling to estimate the probability that the *average* player has a career batting average below 0.300

In [76]:
# TODO
result =analyze_player("General ", alpha0,beta0, 5000)
is_lower_than_averge = [1 if sample < .3  else 0 for sample in result['samples'] ]
print(f"Percentage of the lower than average of 0.3 of all the samples is the {sum(is_lower_than_averge)*100/len(is_lower_than_averge)} %", )

General 

Mean: 0.2579471718657491 
Variance: 0.0006043403373404829 
Std Dev: 0.02458333454477815


.10 quantile :  0.22629470090863318
.90 quantile :  0.29016365309965336
Percentage of the lower than average of 0.3 of all the samples is the 95.14 %


5d. How would you use the estimates from 4b and 4c to describe how good a hitter Hank Aaron was?

TODO: 
???5b and 5c?

We can say that Aaron is considered excellent as only 11.72% of the times his batting average is below .3 which is considered excellent. He is performing above average

6. Find two rows of the `career` dataframe that have the same `eb` estimate (to the third decimal place). Compare the posterior error probability that each of these players has a batting average below 0.300. Reconcile any differences.

## Submission

Download completed **Week6_Statistical_Inference_With_Python_Homework.ipynb** from Google Colab and commit to your personal Github repo you shared with the faculty.